<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/Summirizer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pytextrank

In [ ]:
#pip show pytextrank

In [ ]:
#pip show spacy

In [ ]:
#pip install pyspark

# Importing necessary libraries

In [2]:
import spacy
import pytextrank
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import nltk
import gensim
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages


In [3]:
dataset=pd.read_excel('ARPs_and_ProgrammingPosts.xlsx')

In [ ]:
dataset.head()

,Post_id,Question_title,Question_body,Answer_body,Tokens,W2V_Vect,Tf-IDF_vect,BoW_vect,fastText_vect,GloVe_vect
0,78422247,General architecture for React calling AWS and...,Kinda new to AWS. I have this high-level quest...,"You send a request, you get a response. In ord...",NaN,NaN,NaN,NaN,NaN,NaN
1,78414124,Can Spring MVC be used just for designing clie...,I have some spring boot microservices and I wa...,<blockquote>\ntl;dr: Spring MVC will not contr...,NaN,NaN,NaN,NaN,NaN,NaN
2,78411308,Implementing Data Source Selection Logic in Cl...,I'm trying to properly design an application a...,Determining the source of the information is b...,NaN,NaN,NaN,NaN,NaN,NaN
3,78409121,.NET8 Auth - what should be the architecture f...,I heard that for .NET8 Microsoft gifted us wit...,I have always asked myself this very same ques...,NaN,NaN,NaN,NaN,NaN,NaN
4,78400737,AWS API gateway with Kubernetes services,"I am trying to learn AWS services, and now it ...","Short answer is: no, you don't have to but you...",NaN,NaN,NaN,NaN,NaN,NaN


# Load a spaCy Pipeline Model

---



In [ ]:
# load a spaCy model, depending on language, scale, etc.
import spacy
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

In [ ]:
title_example = """ Framework/Protocol/Tools to use for implementing request-response type application in SOA environment [closed] """

docBodyTosummirize = """Exploring SOA, I come across the two prominent options SOAP and REST. As I mention, I have to make a choice interms a communication methodology to use internally between the modules/layers in my application.
        Is there an obvious/popular choice to make between them when looking for quick request-response and high scalability, load balancing, failover scenario? I don't intend to convert this post into REST vs. SOAP discussion """

docAnswerToSummirize = """if you are probably trying to achieve almost zero application latency, then you must cache almost everything
        near the web layer, like memcached, and postpone or make updates asynchronously. consider also using REST, it demands less overhead than SOAP, and you can use JSON as a document format which
        is also more compact than XML, lowering network throughput requirements """

In [ ]:
#Printing orginal question body text size
print("Original Question text Document Size:",len(docBodyTosummirize), ", while sammarized question text is:",len(docBodyTosummirize))
docBodyTosummirize  = nlp(docBodyTosummirize)

for sent in docBodyTosummirize._.textrank.summary(limit_sentences=4):
    print(sent)
print("-----------------------")

#Printing orginal answer body text size
print("Original Answer Body text Size:",len(docAnswerToSummirize), ", while sammarized answer text is:",len(docAnswerToSummirize))
docAnswerToSummirize = nlp(docAnswerToSummirize)

for sent in docAnswerToSummirize._.textrank.summary(limit_sentences=4):
    print(sent)

In [ ]:
# examine the top-ranked phrases in the document
for phrase in docBodyTosummirize._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)

In [ ]:
# Analyze syntax/tokenization
print("Noun phrases:", [chunk.text for chunk in docBodyTosummirize.noun_chunks])
print("------------------------------------------------------------------------")
print("Verbs:", [token.lemma_ for token in docBodyTosummirize if token.pos_ == "VERB"])

In [ ]:
postId = dataset.Post_id
postId.head()

0    78422247
1    78414124
2    78411308
3    78409121
4    78400737
Name: Post_id, dtype: int64

In [8]:
#https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe?rq=2

#title = dataset.Question_title
title = dataset.iloc[:,1]
title.head()

0    General architecture for React calling AWS and...
1    Can Spring MVC be used just for designing clie...
2    Implementing Data Source Selection Logic in Cl...
3    .NET8 Auth - what should be the architecture f...
4             AWS API gateway with Kubernetes services
Name: Question_title, dtype: object

In [9]:
question_body = dataset.Question_body
#question_body = dataset.iloc[:,2]
question_body.head()

0    Kinda new to AWS. I have this high-level quest...
1    I have some spring boot microservices and I wa...
2    I'm trying to properly design an application a...
3    I heard that for .NET8 Microsoft gifted us wit...
4    I am trying to learn AWS services, and now it ...
Name: Question_body, dtype: object

In [10]:
answer_body = dataset.iloc[:,3]
answer_body.head()

0    You send a request, you get a response. In ord...
1    <blockquote>\ntl;dr: Spring MVC will not contr...
2    Determining the source of the information is b...
3    I have always asked myself this very same ques...
4    Short answer is: no, you don't have to but you...
Name: Answer_body, dtype: object

In [11]:
dataset.Post_id.iloc[0:4]



0    78422247
1    78414124
2    78411308
3    78409121
Name: Post_id, dtype: int64

# Architectural_issue pair

In [5]:
#def  summirizer():
from colorama import Fore

In [6]:
#https://www.geeksforgeeks.org/how-to-iterate-over-rows-and-columns-in-pyspark-dataframe/
#https://www.geeksforgeeks.org/loop-or-iterate-over-all-or-certain-columns-of-a-dataframe-in-python-pandas/
#https://stackoverflow.com/questions/60721458/streaming-output-truncated-to-the-last-5000-lines
for index, row in dataset.iterrows():
    print(f"{Fore.BLACK}Id:{Fore.WHITE}", row[0], f"{Fore.RED}Title:{Fore.WHITE}", row[1], "\n",
          f"{Fore.GREEN}Body:{Fore.WHITE}", "\n",
          row[2], f"{Fore.YELLOW}Answer:{Fore.WHITE}", row[3])
    print("----------------------------------------------------------------------------")

Streaming output truncated to the last 5000 lines.
 Body: 
 While fleshing out a hypothetical domain model, I have found myself wondering whether the better approach in relating domain objects would be to have a parent domain object contain a pointer (the identifier of the child) or if it would be a better approach to use the child objects to build a composite within the parent object.

I can see the pros and cons of each approach, mostly a trade off of size versus complexity, being what it boils down to.  I would tend to lean more towards a identifier relationship approach, as I am not anticipating the need for doing any sort of lazy loading.

Although not directly related, the domain objects are simple POCOs (.NET-equivalent of POJOs).  They are explicitly marked as serializable, as there is a good chance that they will eventually cross between application domains.  LINQ makes the relational identifier approach viable, in my opinion, and I would not consider it at all if LINQ were no

In [ ]:
for index in [0:3]:
  print(title)